In [2]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np

2023-03-17 16:59:31.583468: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 16:59:31.689441: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-17 16:59:31.997891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yramakrishna/anaconda3/lib/python3.9/site-packages/cv2/../../lib64::/home/yramakrishna/anaconda3/envs/tf/lib/
2023-03-17 16:59:31.997940: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Coul

Loading DLC 2.3.1...


In [ ]:
#cropping video and Gamma correction

# Load the video file
video = cv2.VideoCapture(dsv)

# Get the frame rate and total number of frames
fps = int(video.get(cv2.CAP_PROP_FPS))
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

#*********************************************

# Set the start and end points to crop out
start_sec = 10  # Crop out the first x seconds
end_sec = 10 # Crop out the last y seconds

#Set gamma correction value
gamma = 2

#**********************************************

start_frame = int(start_sec * fps)
end_frame = int((total_frames / fps) - end_sec) * fps

# Set the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('CropGr.mp4', fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# Set the current frame number to the start frame
video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# Loop through the frames and write them to the output video
for i in range(start_frame, end_frame):
    ret, frame = video.read()
    if not ret:
        break
    
    # Normalize the pixel values to the range [0, 1]
    frame_normalized = frame.astype(np.float32) / 255.0

    # Apply gamma correction
    frame_corrected = np.power(frame_normalized, 1/gamma)

    # Scale the pixel values back to the range [0, 255]
    frame_scaled = (frame_corrected * 255.0).astype(np.uint8)

    out.write(frame_scaled)

# Release the video objects
video.release()
out.release()
cv2.destroyAllWindows()

In [3]:
os.mkdir(final_sample)
os.chdir('/final_sample')

NameError: name 'final_sample' is not defined

In [ ]:
file = '/final_sample/CropGr.mp4'

bodyparts = ['Eyes_MidPoint', 'RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
             'NostrilsTop_Centre', 'OutlineTop_Mid', 'RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
video_down = file

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = 'primate_face'

In [ ]:
project_name = 'DLC_GazeXBI'
your_name = 'anc'


config_path, train_config_path = deeplabcut.create_pretrained_project(
    project_name,
    your_name,
    video_down,
    videotype=videotype,
    model=model_selection,
    analyzevideo=True,
    createlabeledvideo=False,
    copy_videos=True,
)


In [ ]:
edits = {
    'dotsize': 2,  # size of the dots!
    'pcutoff': 0.95,  # the higher, the more conservative the plotting!
}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

project_path = os.path.dirname(config_path)
full_video_path = os.path.join(
    project_path,
    'videos',
    os.path.basename(video_down),
)

# filter predictions (should already be done above ;):
deeplabcut.filterpredictions(config_path, [full_video_path], videotype=videotype)

# re-create the video with your edits!
deeplabcut.create_labeled_video(config_path, [full_video_path], videotype=videotype, displayedbodyparts=bodyparts,
                                filtered=True)


In [ ]:
#Change the date on the file-------------------------------|---------------- to the current date
#                                                          v
# Input video path
save_path = "/content/final_sample/DLC_GazeXBI-anc-2023-03-15/videos/CropGrDLC_resnet50_DLC_GazeXBIMar15shuffle1_1030000_filtered_labeled.mp4"

# Compressed video path
compressed_path = "/content/final_sample/DLC_GazeXBI-anc-2023-03-15/videos/CropGrDLC_resnet50_DLC_GazeXBIMar15shuffle1_1030000_filtered_labeled_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=320 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)